In [ ]:
# lo que hace es modificar la lista de rutas de búsqueda de módulos de Python (sys.path) para incluir las carpetas ../scripts y ../datasets como ubicaciones adicionales donde Python puede buscar módulos o paquetes cuando hacés un import.
import sys
sys.path.append("../scripts")
sys.path.append("../datasets")

# ================== Evaluación simple estilo LD-SMOTE ==================
# Clasificador fijo: RandomForest(n_estimators=100), sin búsqueda de hiperparámetros.
# Validación: StratifiedKFold (por defecto 10 folds como referencia típica).
# Métricas: Accuracy, F1-macro, G-mean y AUC macro-OVR.

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from imblearn.metrics import geometric_mean_score
import numpy as np

from pc_smote import PCSMOTE  

In [ ]:
def aplicar_pcsmote_y_devolver_resample(X_train, y_train):
    # Llamá a tu clase PC-SMOTE (modo base/full/diagnóstico)
    X_res, y_res = pcsmote.fit_resample_multiclass(X_train, y_train)
    return X_res, y_res

def aplicar_smote_base_y_devolver_resample(X_train, y_train):
    from imblearn.over_sampling import SMOTE
    sobremuestreador = SMOTE(random_state=123)
    X_res, y_res = sobremuestreador.fit_resample(X_train, y_train)
    return X_res, y_res

def sin_sobremuestreo(X_train, y_train):
    # Para comparar contra “base”: devuelve datos tal cual
    return X_train, y_train


In [ ]:
def evaluar_tecnica_con_bosque_aleatorio_sin_tuning(
    matriz_features_completa,
    vector_etiquetas_completo,
    funcion_sobremuestreo_que_devuelve_Xres_yres,
    cantidad_de_folds=10,
    semilla_aleatoria_inicial=123,
    calcular_auc_macro_ovr=True
):
    """
    Ejecuta entrenamiento y evaluación al estilo LD-SMOTE:
    - Clasificador fijo: RandomForest con 100 árboles (sin tuning).
    - CV estratificado con 'cantidad_de_folds'.
    - Calcula Accuracy, F1-macro, G-mean y AUC macro-OVR (si hay proba).

    La 'funcion_sobremuestreo_que_devuelve_Xres_yres' debe aceptar (X_train, y_train)
    y retornar (X_resampleado, y_resampleado), por ejemplo:
        lambda X, y: pcsmote.fit_resample_multiclass(X, y)
    """

    # ----- Hiperparámetros fijos del clasificador (sin tuning) -----
    numero_de_arboles = 100
    profundidad_maxima = None
    usar_bootstrap = True
    cantidad_trabajos_en_paralelo = 1  # mantener 1 para reproducibilidad y simpleza
    usar_oob = False

    # ----- Acumuladores de métricas en todos los folds -----
    lista_accuracy_por_fold = []
    lista_f1_macro_por_fold = []
    lista_gmean_por_fold = []
    lista_auc_macro_por_fold = []

    # ----- Configuración del K-Fold estratificado -----
    validador_estratificado = StratifiedKFold(
        n_splits=cantidad_de_folds,
        shuffle=True,
        random_state=semilla_aleatoria_inicial
    )

    # ----- Bucle de validación -----
    indice_fold_actual = 0
    for indices_entrenamiento, indices_validacion in validador_estratificado.split(
        matriz_features_completa, vector_etiquetas_completo
    ):
        # Partición de datos
        X_entrenamiento = matriz_features_completa[indices_entrenamiento]
        y_entrenamiento = vector_etiquetas_completo[indices_entrenamiento]
        X_validacion = matriz_features_completa[indices_validacion]
        y_validacion = vector_etiquetas_completo[indices_validacion]

        # Sobremuestreo (estilo LD-SMOTE: aplicar técnica y luego entrenar RF fijo)
        X_resampleado, y_resampleado = funcion_sobremuestreo_que_devuelve_Xres_yres(
            X_entrenamiento, y_entrenamiento
        )

        # Clasificador fijo
        clasificador_bosque_aleatorio = RandomForestClassifier(
            n_estimators=numero_de_arboles,
            max_depth=profundidad_maxima,
            bootstrap=usar_bootstrap,
            oob_score=usar_oob,
            n_jobs=cantidad_trabajos_en_paralelo,
            random_state=semilla_aleatoria_inicial + indice_fold_actual
        )

        # Entrenamiento
        clasificador_bosque_aleatorio.fit(X_resampleado, y_resampleado)

        # Predicción discreta (para Accuracy, F1-macro, G-mean)
        y_prediccion_discreta = clasificador_bosque_aleatorio.predict(X_validacion)

        # Métricas principales
        valor_accuracy = accuracy_score(y_validacion, y_prediccion_discreta)
        valor_f1_macro = f1_score(y_validacion, y_prediccion_discreta, average="macro")
        valor_gmean = geometric_mean_score(y_validacion, y_prediccion_discreta, average="macro")

        lista_accuracy_por_fold.append(valor_accuracy)
        lista_f1_macro_por_fold.append(valor_f1_macro)
        lista_gmean_por_fold.append(valor_gmean)

        # AUC macro-OVR (si corresponde)
        if calcular_auc_macro_ovr:
            try:
                y_probabilidades = clasificador_bosque_aleatorio.predict_proba(X_validacion)
                valor_auc_macro = roc_auc_score(
                    y_validacion, y_probabilidades, multi_class="ovr", average="macro"
                )
                lista_auc_macro_por_fold.append(valor_auc_macro)
            except Exception:
                # Si por algún motivo no se puede calcular AUC, no interrumpo la corrida
                pass

        indice_fold_actual = indice_fold_actual + 1

    # ----- Resumen (promedio y desvío estándar) -----
    resultados_resumen = {
        "cv_accuracy_promedio": float(np.mean(lista_accuracy_por_fold)) if len(lista_accuracy_por_fold) > 0 else None,
        "cv_accuracy_desvio": float(np.std(lista_accuracy_por_fold)) if len(lista_accuracy_por_fold) > 0 else None,

        "cv_f1_macro_promedio": float(np.mean(lista_f1_macro_por_fold)) if len(lista_f1_macro_por_fold) > 0 else None,
        "cv_f1_macro_desvio": float(np.std(lista_f1_macro_por_fold)) if len(lista_f1_macro_por_fold) > 0 else None,

        "cv_gmean_promedio": float(np.mean(lista_gmean_por_fold)) if len(lista_gmean_por_fold) > 0 else None,
        "cv_gmean_desvio": float(np.std(lista_gmean_por_fold)) if len(lista_gmean_por_fold) > 0 else None,

        "cv_auc_macro_ovr_promedio": float(np.mean(lista_auc_macro_por_fold)) if len(lista_auc_macro_por_fold) > 0 else None,
        "cv_auc_macro_ovr_desvio": float(np.std(lista_auc_macro_por_fold)) if len(lista_auc_macro_por_fold) > 0 else None,

        "cantidad_folds_ejecutados": len(lista_accuracy_por_fold)
    }

    return resultados_resumen


In [ ]:
# Adaptador mínimo para tu técnica (ejemplos):

def aplicar_pcsmote_y_devolver_resample(X_train, y_train):
    # Reemplazar por tu llamada real (modo base/full/diagnóstico)
    X_res, y_res = pcsmote.fit_resample_multiclass(X_train, y_train)
    return X_res, y_res

def aplicar_smote_base_y_devolver_resample(X_train, y_train):
    from imblearn.over_sampling import SMOTE
    sobremuestreador = SMOTE(random_state=123)
    X_res, y_res = sobremuestreador.fit_resample(X_train, y_train)
    return X_res, y_res

# Ejecución “simple, rápida y comparable”
resultados_pcsmote = evaluar_tecnica_con_bosque_aleatorio_sin_tuning(
    X, y,
    funcion_sobremuestreo_que_devuelve_Xres_yres=aplicar_pcsmote_y_devolver_resample,
    cantidad_de_folds=10,           # 10-fold CV
    semilla_aleatoria_inicial=123,
    calcular_auc_macro_ovr=True
)

resultados_smote = evaluar_tecnica_con_bosque_aleatorio_sin_tuning(
    X, y,
    funcion_sobremuestreo_que_devuelve_Xres_yres=aplicar_smote_base_y_devolver_resample,
    cantidad_de_folds=10,
    semilla_aleatoria_inicial=123,
    calcular_auc_macro_ovr=True
)

print("PC-SMOTE:", resultados_pcsmote)
print("SMOTE   :", resultados_smote)
